In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.0,
    max_retries=2,
    # other params...
)

response = llm.invoke("The first person to land on moon was...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/software-engineer-i-itc/job/R-71708")
page_data = loader.load().pop().page_content
print(page_data)





















Software Engineer I, ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Se

In [73]:
from langchain_core.prompts import ChatPromptTemplate
prompt_extract = ChatPromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE
    {page_data}

    ### TASK
    Extract all job postings.

   ### OUTPUT FORMAT (STRICT)
   Return a single valid JSON object with these keys:
   - "role"
   - "experience"
   - "skills"
   - "description"

   Do not wrap it in an array.
   Do not add any text outside the JSON.

    ### RULES
    - Keep the **original phrasing of skills as much as possible**.
    - Group related terms together logically (e.g., `"AWS - EC2, ECS, API Gateway"`, not `"AWS"`, `"EC2"`, `"ECS"`, ... separately).
    - If a value is missing/unclear, use an empty string "" (or [] for "skills").
    - Keep JSON valid: no trailing commas, no comments, no code fences.
    - No explanations or text outside the JSON.

    ### VALID JSON (NO PREAMBLE)

    """
    )
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})
print(res.content)


{
  "role": "Software Engineer I, ITC",
  "experience": "0-2 years software development experience",
  "skills": [
    "React",
    "TypeScript",
    "microservices architecture",
    "domain driven design",
    "RESTful API's",
    "Node.js",
    "cloud services",
    "AWS - EC2, ECS, API Gateway, Lambda, DynamoDB",
    "CI/CD",
    "DevOps",
    "Scrum",
    "Agile",
    "E2E testing frameworks",
    "database management",
    "performance optimization",
    "continuous integration/continuous deployment (CI/CD) pipelines"
  ],
  "description": "Build and deliver the technical runway for existing and new technological capabilities to support multiple mission critical systems within the Paid Media space."
}


In [65]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer I, ITC',
 'experience': '0-2 years software development experience',
 'skills': ['React',
  'TypeScript',
  'microservices architecture',
  'domain driven design',
  "RESTful API's",
  'Node.js',
  'cloud services',
  'AWS - EC2, ECS, API Gateway, Lambda, DynamoDB',
  'CI/CD',
  'DevOps',
  'Scrum',
  'Agile principles',
  'E2E testing frameworks',
  'database management',
  'performance optimization',
  'continuous integration/continuous deployment (CI/CD) pipelines'],
 'description': 'Build and deliver the technical runway for existing and new technological capabilities to support multiple mission critical systems within the Paid Media space. You will rely on your understanding of scalable, highly available, and consistent distributed systems to design world class solutions.'}

In [66]:
type(json_res)

dict

In [67]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [68]:
import chromadb
import uuid

client = chromadb. PersistentClient('vectorstore') #PersistentClient will create db on disk client will on memory
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
   for _, row in df. iterrows():
       collection.add(documents=row["Techstack"],
                      metadatas={"links": row["Links"]},
                      ids=[str(uuid.uuid4())])

In [71]:
links = collection.query(query_texts= job['skills'],n_results=2).get('metadatas')
print(links)

[[{'links': 'https://example.com/react-portfolio'}, {'links': 'https://example.com/react-native-portfolio'}], [{'links': 'https://example.com/typescript-frontend-portfolio'}, {'links': 'https://example.com/full-stack-js-portfolio'}], [{'links': 'https://example.com/xamarin-portfolio'}, {'links': 'https://example.com/android-portfolio'}], [{'links': 'https://example.com/kotlin-backend-portfolio'}, {'links': 'https://example.com/android-portfolio'}], [{'links': 'https://example.com/wordpress-portfolio'}, {'links': 'https://example.com/magento-portfolio'}], [{'links': 'https://example.com/react-portfolio'}, {'links': 'https://example.com/full-stack-js-portfolio'}], [{'links': 'https://example.com/xamarin-portfolio'}, {'links': 'https://example.com/wordpress-portfolio'}], [{'links': 'https://example.com/flutter-portfolio'}, {'links': 'https://example.com/react-portfolio'}], [{'links': 'https://example.com/magento-portfolio'}, {'links': 'https://example.com/wordpress-portfolio'}], [{'links'

In [70]:
job = json_res
job['skills']

['React',
 'TypeScript',
 'microservices architecture',
 'domain driven design',
 "RESTful API's",
 'Node.js',
 'cloud services',
 'AWS - EC2, ECS, API Gateway, Lambda, DynamoDB',
 'CI/CD',
 'DevOps',
 'Scrum',
 'Agile principles',
 'E2E testing frameworks',
 'database management',
 'performance optimization',
 'continuous integration/continuous deployment (CI/CD) pipelines']

In [72]:
prompt_email = ChatPromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
        
    ### INSTRUCTION:
    You are Siddharth, a business development executive at XYZ. XYZ is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools. 
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
    process optimization, cost reduction, and heightened overall efficiency. 

    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of XYZ 
    in fulfilling their needs.

    You are also given a list of portfolio links: {link_list}
    Use these links in the email naturally, for example:
    - “Here are a few relevant projects: [link]”
    - “You can explore a similar solution here: [link]”
    DO NOT make up any client names or project details. Just use the links as they are.

    Remember:
    - Do not include placeholders like [Client Name].
    - Keep the tone professional and warm.
    - End with a clear call to action.

    ### EMAIL (NO PREAMBLE):

    """
    )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Solutions for Your Paid Media Systems

Dear Hiring Manager,

I came across the job description for a Software Engineer I, ITC at your organization, and I was impressed by the requirements. At XYZ, we specialize in providing tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency.

Our team has extensive experience in designing and delivering world-class solutions for mission-critical systems. We have a proven track record of leveraging scalable, highly available, and consistent distributed systems to meet the needs of our clients.

Here are a few relevant projects that demonstrate our expertise:

- We have successfully implemented React-based solutions for various clients, which can be seen in our React portfolio: https://example.com/react-portfolio
- Our team has also developed full-stack JavaScript solutions, which can be explored in our full-stack JS portfolio: https://example.com/full-stack-js-portfolio
- W